In [1]:
import os

In [2]:
%pwd

'/Users/adityakumar/BankCD-Prediction/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/adityakumar/BankCD-Prediction'

Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str
    subsample: float
    reg_lambda: int
    reg_alpha: int
    n_estimators: int
    max_depth: int
    learning_rate: float
    gamma: int
    colsample_bytree: float
    eval_metric: str
    random_state: int


Config

In [6]:
from src.bankcdProject.constant import *
from src.bankcdProject.utils.common import read_yaml,create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBoost  
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            target_column=schema.name,
            
            # XGBoost parameters
            subsample=params.subsample,
            reg_lambda=params.reg_lambda,
            reg_alpha=params.reg_alpha,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            learning_rate=params.learning_rate,
            gamma=params.gamma,
            colsample_bytree=params.colsample_bytree,
            eval_metric=params.eval_metric,
            random_state=params.random_state
        )

        return model_trainer_config


Compoennts

In [10]:
import pandas as pd
import os
from src.bankcdProject import logger
from xgboost import XGBClassifier
import joblib

[2024-11-13 14:03:34,460: INFO: utils: NumExpr defaulting to 8 threads.]


In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        xgb_model = XGBClassifier(
            subsample=self.config.subsample,
            reg_lambda=self.config.reg_lambda,
            reg_alpha=self.config.reg_alpha,
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            learning_rate=self.config.learning_rate,
            gamma=self.config.gamma,
            colsample_bytree=self.config.colsample_bytree,
            eval_metric=self.config.eval_metric,
            random_state=self.config.random_state
        )
        xgb_model.fit(train_x, train_y)

        joblib.dump(xgb_model, os.path.join(self.config.root_dir, self.config.model_name))

Configuration

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    print("An error occurred during model training:", e)
    raise e


[2024-11-13 14:08:39,529: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-13 14:08:39,535: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-13 14:08:39,537: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-13 14:08:39,538: INFO: common: created directory at: artifacts]
[2024-11-13 14:08:39,539: INFO: common: created directory at: artifacts/model_trainer]
